<center><img src="./logo.png" alt="Header" style="width: 800px;"/></center>

# 定义手势识别函数部分 Define the gesture recognition function part

In [ ]:
#导入相关的模块   Import related modules
import threading
import cv2 
import time
import math
from time import sleep
import ipywidgets.widgets as widgets
# 导入oled屏幕库 Import oled screen library
import sys
sys.path.append('/home/pi/software/oled_yahboom/')
from yahboom_oled import *
# 创建oled对象 Create an oled object
oled = Yahboom_OLED(debug=False)

from gesture_action import handDetector

In [ ]:
g_camera = cv2.VideoCapture(0)
g_camera.set(3, 320)
g_camera.set(4, 240)
g_camera.set(5, 30)  #设置帧率 Setting the frame rate
# g_camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# g_camera.set(cv2.CAP_PROP_BRIGHTNESS, 40) #设置亮度 -64 - 64  0.0 Set Brightness -64 - 64 0.0
# g_camera.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0 Set Contrast -64 - 64 2.0
# g_camera.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000  156.0 Set the exposure value 1.0 - 5000 156.0


# 定义摄像头显示组件 Define the camera display component

In [ ]:
hand_detector = handDetector(detectorCon=0.75)
image_original = widgets.Image(format='jpeg', width=640, height=480)
image_result = widgets.Image(format='jpeg', width=640, height=480)
image_widget = widgets.HBox([image_original, image_result])

In [ ]:
#bgr8转jpeg格式 bgr8 to jpeg format
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# 主进程 Main Process

In [ ]:
display(image_widget)   
try:
    oled.init_oled_process() #初始化oled进程 Initialize oled process
    while True:
        ret, frame = g_camera.read()
        frame, img = hand_detector.findHands(frame, draw=False)
        if len(hand_detector.lmList) != 0:
            finger_number = hand_detector.get_gesture()
            cv2.rectangle(frame, (0, 430), (230, 480), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, str(finger_number), (10, 470), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
            finger_str=f"Number:{finger_number}"
            print(finger_str)
            oled.clear()
            oled.add_line("Gesture_Reco",1)
            oled.add_line(finger_str, 3)
            oled.refresh()
        else:
            oled.clear()
            oled.add_line("Gesture_Reco",1)
            oled.add_line("Number:None",3)
            oled.refresh()
        image_original.value = bgr8_to_jpeg(frame)
        image_result.value = bgr8_to_jpeg(img)
except:
    print(" Program closed! ")
    # 恢复屏幕基础数据显示 Restore basic data display on screen
    os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")
    pass


In [ ]:
g_camera.release()              #使用完成对象记住释放掉对象，不然下一个程序使用这个对象模块会被占用，导致无法使用  Release resources